## Step 1 Fetch Stock Prices
We will use the NVIDIA and CocaCola stock historical data for the analysis, as well as getting the historical data of the NASDAQ market. The stock history period ranges from 2023-01-01 to 2024-01-01

In [1]:
import yfinance as yf

In [2]:
# Define the stock symbol and the time period
nvidia_stock_symbol = "NVDA"  # NVIDIA Ltd.
coca_stock_symbol = "KO"  # Coca Cola Ltd.
market_symbol = "^GSPC"  # NASDAQ 100

start_date = "2023-01-01"
end_date = "2024-01-01"

In [3]:
# Fetch daily historical stock data
nvidia_stock_data = yf.download(nvidia_stock_symbol, start=start_date, end=end_date)
coca_stock_data = yf.download(coca_stock_symbol, start=start_date, end=end_date)
market_data = yf.download(market_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
nvidia_stock_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,148.509995,149.960007,140.960007,143.149994,143.079788,40127700
2023-01-04,145.669998,148.529999,142.410004,147.490005,147.417679,43132400
2023-01-05,144.910004,145.639999,141.479996,142.649994,142.580032,38916800
2023-01-06,144.740005,150.100006,140.339996,148.589996,148.517136,40504400
2023-01-09,152.839996,160.559998,151.410004,156.279999,156.203369,50423100
...,...,...,...,...,...,...
2023-12-22,491.950012,493.829987,484.670013,488.299988,488.277069,25213900
2023-12-26,489.679993,496.000000,489.600006,492.790009,492.766907,24420000
2023-12-27,495.109985,496.799988,490.850006,494.170013,494.146820,23364800


In [5]:
# Extract the closing prices
nvidia_adj_close = nvidia_stock_data["Adj Close"].pct_change().dropna()
coca_adj_close = coca_stock_data["Adj Close"].pct_change().dropna()
market_adj_close = market_data["Adj Close"].pct_change().dropna()

# verify the integrity of the closing price data
print(nvidia_adj_close.shape)
print(coca_adj_close.shape)
print(market_adj_close.shape)

(249,)
(249,)
(249,)


## Step 2 Calculate the Beta Coefficient
Recall that the beta coefficient
$$\beta=\dfrac{Cov(R_e, R_m)}{Var(R_m)}$$
where $R_e$ is the return on an individual stock and $R_m$ is the return of the overall market.

In [6]:
import numpy as np

In [7]:
covariance_nvidia = np.cov(nvidia_adj_close, market_adj_close)[0, 1]
covariance_coca = np.cov(coca_adj_close, market_adj_close)[0, 1]
variance_market = np.var(market_adj_close)

In [8]:
beta_nvidia = covariance_nvidia / variance_market
beta_coca = covariance_coca / variance_market

In [9]:
print(f"The beta of {nvidia_stock_symbol} in relation to {market_symbol} is: {beta_nvidia:.4f}")
print(f"The beta of {coca_stock_symbol} in relation to {market_symbol} is: {beta_coca:.4f}")

The beta of NVDA in relation to ^GSPC is: 2.0483
The beta of KO in relation to ^GSPC is: 0.3820


(TODO: 在这边讨论beta值对股票性能的体现)

## Step 3 Estimates the CAPM model
Recall that in the capital asset pricing model (CAPM):
$$E(R_i)=R_f+\beta_i(E(R_m)-R_f)$$
where $E(R_i)$ is the capital asset expected return, $R_f$ is the risk-free rate of interest,  $\beta_i$ is the sensitivity and $E(R_m)$ is the expected return of the market

In [10]:
from scipy.stats import linregress

In [11]:
# handle the average risk-free rate
risk_free_rate_data = yf.download('^IRX', start=start_date, end=end_date)
risk_free_rate = risk_free_rate_data['Adj Close'] / 100  # Convert percentage to decimal
avg_risk_free_rate = np.mean(risk_free_rate)

avg_risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05045160009384155

In [12]:
# handle the estimated beta value
est_beta_nvidia = linregress(market_adj_close, nvidia_adj_close)[0]
est_beta_coca = linregress(market_adj_close, coca_adj_close)[0]

est_beta_nvidia

2.040046303908817

In [13]:
# handle the CAPM output
def calculate_CAPM(expected_market_return, risk_free_rate, beta):
    return risk_free_rate + beta * (expected_market_return - risk_free_rate)


expected_market_return = 0.2
expected_nvidia_return = calculate_CAPM(expected_market_return, avg_risk_free_rate, est_beta_nvidia)
expected_coca_return = calculate_CAPM(expected_market_return, avg_risk_free_rate, est_beta_coca)

In [14]:
print(
    f'NVIDIA: Calculated beta: {beta_nvidia:.4f}; Estimated beta: {est_beta_nvidia:.4f}; Expected return (CAPM): {expected_nvidia_return:.4f}')
print(
    f'CocaCola: Calculated beta: {beta_coca:.4f}; Estimated beta: {est_beta_coca:.4f}; Expected return (CAPM): {expected_coca_return:.4f}')

NVIDIA: Calculated beta: 2.0483; Estimated beta: 2.0400; Expected return (CAPM): 0.3555
CocaCola: Calculated beta: 0.3820; Estimated beta: 0.3805; Expected return (CAPM): 0.1074


(TODO: 在这边讨论区别)

## Step 4 Minimum Variance Portfolio

In [15]:
# Calculate the covariance matrix between two stocks and invert the covariance matrix
cov_matrix = np.cov(nvidia_adj_close, coca_adj_close)
inv_cov_matrix = np.linalg.inv(cov_matrix)

# define the expected return rate
expected_return_rates = [0.35, 0.1]

# Calculate the weights for the minimum variance portfolio
weights_min_variance = inv_cov_matrix @ expected_return_rates / np.sum(inv_cov_matrix @ expected_return_rates)

print("Minimum Variance Portfolio Weights:")
print(weights_min_variance)

Minimum Variance Portfolio Weights:
[0.21248442 0.78751558]


In [16]:
# Build the portfolio
w1, w2 = weights_min_variance
portfolio_adj_close = w1 * nvidia_adj_close + w2 * coca_adj_close
portfolio_adj_close.shape

(249,)

In [17]:
# calculate the CAPM for the built portfolio
est_beta_portfolio = linregress(market_adj_close, portfolio_adj_close)[0]
expected_market_return = 0.2
expected_portfolio_return = calculate_CAPM(expected_market_return, avg_risk_free_rate, est_beta_portfolio)

print(f'Minimum Variance Portfolio Expected Return (CAPM): {expected_portfolio_return:.4f}')

Minimum Variance Portfolio Expected Return (CAPM): 0.1601


## Step 5 Compare to the weekly or monthly stock price

In [18]:
# Fetch monthly historical stock data
M_nvidia_stock_data = yf.download(nvidia_stock_symbol, start=start_date, end=end_date, interval="1mo")
M_coca_stock_data = yf.download(coca_stock_symbol, start=start_date, end=end_date, interval="1mo")
M_market_data = yf.download(market_symbol, start=start_date, end=end_date, interval="1mo")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [19]:
M_nvidia_stock_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-01,148.509995,206.279999,140.339996,195.369995,195.274200,945496000
2023-02-01,196.910004,238.880005,196.110001,232.160004,232.046143,1039347900
2023-03-01,231.919998,278.339996,222.970001,277.769989,277.633759,1126373100
2023-04-01,275.089996,281.100006,262.200012,277.489990,277.401031,743592100
2023-05-01,278.399994,419.380005,272.399994,378.339996,378.218689,1169636000
2023-06-01,384.890015,439.899994,373.559998,423.019989,422.884369,1052209200
2023-07-01,425.170013,480.880005,413.459991,467.290009,467.188538,870489500
2023-08-01,464.600006,502.660004,403.109985,493.549988,493.442780,1363143600
2023-09-01,497.619995,498.000000,409.799988,434.989990,434.895508,857510100


In [20]:
# Extract the monthly closing prices
M_nvidia_adj_close = M_nvidia_stock_data["Adj Close"].pct_change().dropna()
M_coca_adj_close = M_coca_stock_data["Adj Close"].pct_change().dropna()
M_market_adj_close = M_market_data["Adj Close"].pct_change().dropna()

# verify the integrity of the closing price data
print(M_nvidia_adj_close.shape)
print(M_coca_adj_close.shape)
print(M_market_adj_close.shape)

(11,)
(11,)
(11,)


In [21]:
# handle the montly average risk-free rate
M_risk_free_rate_data = yf.download('^IRX', start=start_date, end=end_date, interval="1mo")
M_risk_free_rate = M_risk_free_rate_data['Adj Close'] / 100  # Convert percentage to decimal
M_avg_risk_free_rate = np.mean(M_risk_free_rate)

M_avg_risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05064416686693828

In [22]:
# handle the estimated beta value for monthly data
M_est_beta_nvidia = linregress(M_market_adj_close, M_nvidia_adj_close)[0]
M_est_beta_coca = linregress(M_market_adj_close, M_coca_adj_close)[0]

print(M_est_beta_nvidia)
print(M_est_beta_coca)

1.102999445347155
0.6498956065709122


In [23]:
# handle the CAPM output
def calculate_CAPM(expected_market_return, risk_free_rate, beta):
    return risk_free_rate + beta * (expected_market_return - risk_free_rate)


expected_market_return = 0.2
M_expected_nvidia_return = calculate_CAPM(expected_market_return, M_avg_risk_free_rate, M_est_beta_nvidia)
M_expected_coca_return = calculate_CAPM(expected_market_return, M_avg_risk_free_rate, M_est_beta_coca)

In [24]:
print("For daily profile:")
print(
    f'NVIDIA: Estimated beta: {est_beta_nvidia:.4f}; Expected return (CAPM): {expected_nvidia_return:.4f}')
print(
    f'CocaCola: Estimated beta: {est_beta_coca:.4f}; Expected return (CAPM): {expected_coca_return:.4f}')

print("For monthly profile:")
print(
    f'NVIDIA: Estimated beta: {M_est_beta_nvidia:.4f}; Expected return (CAPM): {M_expected_nvidia_return:.4f}')
print(
    f'CocaCola: Estimated beta: {M_est_beta_coca:.4f}; Expected return (CAPM): {M_expected_coca_return:.4f}')

For daily profile:
NVIDIA: Estimated beta: 2.0400; Expected return (CAPM): 0.3555
CocaCola: Estimated beta: 0.3805; Expected return (CAPM): 0.1074
For monthly profile:
NVIDIA: Estimated beta: 1.1030; Expected return (CAPM): 0.2154
CocaCola: Estimated beta: 0.6499; Expected return (CAPM): 0.1477
